(numbers)=
# Numbers

## Introduction

In this chapter, you'll learn useful tools for creating and manipulating numeric vectors. We'll start by going into a little more detail of `.count()` before diving into various numeric transformations. You'll then learn about more general transformations that can be applied to other types of column, but are often used with numeric column. Then you'll learn about a few more useful summaries.

### Prerequisites

This chapter mostly uses functions from **pandas**, which you are likely to already have installed bu you can install using `pip install pandas` in the terminal. We'll use real examples from nycflights13, as well as toy examples made with fake data.

Let's first load up the NYC flights data (note that this is not a tiny file so will take a moment to load from the internet)


In [ ]:
import matplotlib_inline.backend_inline
import matplotlib.pyplot as plt

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [ ]:
import pandas as pd

flights = pd.read_parquet("https://github.com/aeturrell/coding-for-economists/blob/main/data/flights.parquet?raw=true")

### Counts

It's surprising how much data science you can do with just counts and a little basic arithmetic, so **pandas** strives to make counting as easy as possible with `.count()` and `.value_counts()`. The former just provides a straight count of all the non NA items:

In [ ]:
flights["dest"].count()

The latter provides a count broken down by type:

In [ ]:
flights["dest"].value_counts()

This is automatically sorted in order of the most common category. You can perform the same computation "by hand" with `group_by()`, `agg()` and then using the count function. This is useful because it allows you to compute other summaries at the same time:

In [ ]:
(
    flights.groupby(["dest"])
    .agg(
        mean_delay=("dep_delay", "mean"),
        count_flights=("dest", "count"),
    )
    .sort_values(by="count_flights", ascending=False)
)

Note that a weighted count is just a sum. For example you could "count" the number of miles each plane flew:

In [ ]:
(flights.groupby("tailnum").agg(miles=("distance", "sum")))

You can count missing values by combining `sum()` and `isnull()`. In the flights dataset this represents flights that are cancelled. Note that because there isn't a simple string name for applying `.isnull` followed by `.sum` (unlike just running `sum`, which would be given by the string "sum"), we need to use a lambda function in the below:

In [ ]:
(flights.groupby("dest").agg(n_cancelled=("dep_time", lambda x: x.isnull().sum())))

## Numeric Transformations

Transformation functions have an output is the same length as the input. The vast majority of transformation functions are either built into Python or come with the numerical package **numpy**. It's impractical to list all the possible numerical transformations so this section will show the most useful ones.

Basic number arithmatic is achieved by `+` (addition), `-` (subtraction), `*` (multiplication), `/` (division), `**` (powers), `%` (modulo), and `@` (tensor product). Most of these functions don't need a huge amount of explanation because you'll be familiar with them already (and you can look up the others when you do need them).

When you have two numeric columns of equal length and you add or subtract them, it's pretty obvious what's going to happen. But we do need to talk about what happens when there is a variable involved that is *not* as long as the column. This is important for operations like `flights.assign(air_time = air_time / 60)` because there are 336,776 numbers on the left of `/` but only one on the right. In this case, **pandas** will understand that you'd like to divide *all* values of air time by 60. This is sometimes called 'broadcasting'. Below is a digram that tries to explain what's going on:

![](https://numpy.org/doc/stable/_images/broadcasting_1.png)

You can find out much more about [broadcasting on the **numpy** documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html). **pandas** is built on top of **numpy** and inherits some of its functionality.  


When operating on two columns, **pandas** compares their shapes element-wise. Two columns are compatible when they are equal, or one of them is a scalar. If these conditions are not met, you will get an error.



### Minimum and Maximum

The arithmetic functions do what you'd expect.

In [ ]:
flights["distance"].max()

Sometimes, you'd like to look at the maximum or minimum value across rows *or* columns. As often is the case with **pandas**, you can specify rows or columns to apply functions to by passing `axis=0` (index) or `axis=1` (columns) to that function. The axis designation can be confusing: remember that you are asking which dimension you wish to aggregate over, leaving you with the other dimension. So if we wish to find the minimum in each row, we aggregate / collapse columns, so we need to pass `axis=1`.

In [ ]:
df = pd.DataFrame({"x": [1, 5, 7], "y": [3, 2, pd.NA]})
df

Now let's find the min by row:

In [ ]:
df.min(axis=1)

### Modular arithmetic

Modular arithmetic is the technical name for the type of math you do on whole numbers, i.e. division that yields a whole number and a remainder. In Python, `//` does integer division and `%` computes the remainder:

In [ ]:
print([x for x in range(1, 11)])
print("divided by 3 gives")
print("remainder:")
print([x % 3 for x in range(1, 11)])
print("divisions:")
print([x // 3 for x in range(1, 11)])

Modular arithmetic is handy for the flights dataset, because we can use it to unpack the `sched_dep_time` variable into and `hour` and `minute`:

In [ ]:
flights.assign(
    hour=lambda x: x["sched_dep_time"] // 100,
    minute=lambda x: x["sched_dep_time"] % 100,
)

### Logarithms

Logarithms are an incredibly useful transformation for dealing with data that ranges across multiple orders of magnitude. They also convert exponential growth to linear growth. For example, take compounding interest --- the amount of money you have at `year + 1` is the amount of money you had at `year` multiplied by the interest rate. That gives a formula like `money = starting * interest ** year`:


In [ ]:
import numpy as np

starting = 100
interest = 1.05
money = pd.DataFrame(
    {"year": 2000 + np.arange(1, 51), "money": starting * interest ** np.arange(1, 51)}
)
money.head()

If you plot this data, you'll get an exponential curve:

In [ ]:
money.plot(x="year", y="money");

Log transforming the y-axis gives a straight line:

In [ ]:
money.plot(x="year", y="money", logy=True);

This a straight line because `log(money) = log(starting) + n * log(interest)` matches the pattern for a line, `y = m * x + b`. This is a useful pattern: if you see a (roughly) straight line after log-transforming the y-axis, you know that there's underlying exponential growth.

If you're log-transforming your data you have a choice of a lot of logarithms provided by **numpy** but there are three ones you'll want to commonly use: assuming you've imported it using `import numpy as np`, you have `np.log()` (the natural log, base e), `np.log2()` (base 2), and `np.log10()` (base 10).

The inverse of `log()` is `np.exp()`; to compute the inverse of `np.log2()` or `np.log10()` you'll need to use `2**` or `10**`.

### Rounding

To round to a specific number of decimal places, use `.round(n)` where `n` is the number of decimal places that you wish to round to.

In [ ]:
money.head().round(2)

This can also be applied to individual columns or differentially to columns via a dictionary:

In [ ]:
money.head().round({"year": 0, "money": 1})

`.round(n)` rounds to the nearest `10**(-n)` so `n = 2` will round to the nearest 0.01. This definition is useful because it implies `.round(-2)` will round to the nearest hundred, which indeed it does:

In [ ]:
money.tail().round({"year": 0, "money": -2})

Sometimes, you'll want to round according to significant figures rather than decimal places. There's not a really easy way to do this but you can define a custom function to do it. Here's an example of rounding to 2 significant figures (change the `2` in the below to round to a different number of significant figures):

In [ ]:
money["money"].head().apply(lambda x: float(f'{float(f"{x:.2g}"):g}'))

If you have an array or list of numbers outside of a data frame, you can use the **numpy** function

In [ ]:
np.round([1.5, 2.5, 1.4])

**numpy** has both `.floor` and `.ceil` methods too

In [ ]:
real_nums = 100 * np.random.random(size=10)
real_nums

In [ ]:
np.ceil(real_nums)

In [ ]:
np.floor(real_nums)

Remember that you can always apply **numpy** functions to **pandas** data frame columns like so:

In [ ]:
money["money"].head().apply(np.ceil)

### Cumulative and Rolling Aggregates

**pandas** has several cumulative functions, including `.cumsum`, `.cummax` and `.cummin`, and `.cumprod`. 

In [ ]:
money["money"].tail().cumsum()

As ever, this can be applied across rows too by passing `axis=1`.

## General transformations

The following sections describe some general transformations that are often used with numeric vectors, but which can be applied to all other column types.

### Ranking

**pandas**' rank function is `.rank`. Let's look back at the data we made earlier and rank it:

In [ ]:
df

In [ ]:
df.rank()

Of course, there's no change here because the items were already ranked! We can also do a pct rank by passing the keyword argument `pct=True`.

In [ ]:
df.rank(pct=True)

### Offsets and Shifting

Offsets allow you to 'roll' columns up and down relative to their original position, ie to offset their location relative to the index. The function that does this is called `shift` and it produces leads or lags depending on whether you use positive or negative values respectively. Remember that a lead is going to shift the pattern in the data to the left when plotted against the index, while the lag is going to shift patterns to the right. Leads and lags are particularly useful for time series data (which we haven't yet seen).

Let's see an example of offsets with the `money` data frame from earlier:

In [ ]:
money["money_lag_5"] = money["money"].shift(5)
money["money_lead_10"] = money["money"].shift(-10)
money.set_index("year").plot();

### Exercises

1.  Find the 10 most delayed flights using a ranking function.

2.  Which plane (`"tailnum"`) has the worst on-time record?

3.  What time of day should you fly if you want to avoid delays as much as possible?

4.  For each destination, compute the total minutes of delay. For each flight, compute the proportion of the total delay for its destination.

5.  Delays are typically temporally correlated: even once the problem that caused the initial delay has been resolved, later flights are delayed to allow earlier flights to leave. Using `.shift()`, explore how the average flight delay for an hour is related to the average delay for the previous hour.

6.  Look at each destination. Can you find flights that are suspiciously fast? (i.e. flights that represent a potential data entry error). Compute the air time of a flight relative to the shortest flight to that destination. Which flights were most delayed in the air?

7.  Find all destinations that are flown by at least two carriers. Use those destinations to come up with a relative ranking of the carriers based on their performance for the same destination.


## More Useful Summary Statistics

We've seen how useful `.mean`, `.count`, and `.value_counts` can be for analysis. **pandas** has a great many more built-in summary statistics functions, however. These include `.median` (you may find it interesting to compare the mean vs the median when looking at the hourly departure delay in the flights data), `.mode`, `.min`, and `.max`.

A class of useful summary statistics are provided by the `.quantile` function, which is the same as `median` for `.quantile(0.5)`. The quantile at x% is the value that x% of values are below. (Note that under this definition, `.quantile(1)` will be the same as `.max()`.) Let's see an example with the 25th percentile.

In [ ]:
money["money"].quantile(0.25)

Sometimes you don't just want one percentile, but a bunch of them. **pandas** makes this very easy by allowing you to pass a list of quantiles:

In [ ]:
money["money"].quantile([0, 0.25, 0.5, 0.75])

### Spread

Sometimes you're not so interested in where the bulk of the data lies, but how it is spread out.
Two commonly used summaries are the standard deviation, `.std()`, and the inter-quartile range, which you can compute from the relevant quantiles, ie it's the quantile at 75% minus the quantile at 25% and gives you the range that contains the middle 50% of the data.

We can use this to reveal a small oddity in the flights data. You might expect that the spread of the distance between origin and destination to be zero, since airports are always in the same place. But the code below makes it looks like one airport, [EGE](https://en.wikipedia.org/wiki/Eagle_County_Regional_Airport), might have moved.

In [ ]:
(
    flights.groupby(["origin", "dest"])
    .agg(
        distance_sd=("distance", lambda x: x.quantile(0.75) - x.quantile(0.25)),
        count=("distance", "count"),
    )
    .query("distance_sd > 0")
)

### Distributions

It's worth remembering that all of the summary statistics described above are a way of reducing the distribution down to a single number. This means that they're fundamentally reductive, and if you pick the wrong summary, you can easily miss important differences between groups. That's why it's always a good idea to visualise the distribution of values as well as using aggregate statistics.

The plot below shows the overall distribution of departure delays. The distribution is so skewed that we have to zoom in to see the bulk of the data. This suggests that the mean is unlikely to be a good summary and we might prefer the median instead.

In [ ]:
flights["dep_delay"].plot.hist(bins=50, title="         Distribution: length of delay");

In [ ]:
flights.query("dep_delay <= 120")["dep_delay"].plot.hist(
    bins=50, title="         Distribution: length of delay"
);

Two histograms of `"dep_delay"`. On the former, it's very hard to see any pattern except that there's a very large spike around zero, the bars rapidly decay in height, and for most of the plot, you can't see any bars because they are too short to see. On the latter, where we've discarded delays of greater than two hours, we can see that the spike occurs slightly below zero (i.e. most flights leave a couple of minutes early), but there's still a very steep decay after that.

Don't be afraid to explore your own custom summaries specifically tailored for the data that you're working with. In this case, that might mean separately summarising the flights that left early vs the flights that left late, or given that the values are so heavily skewed, you might try a log-transformation. Finally, don't forget that it's always a good idea to include the number of observations in each group when creating summaries.

### Exercises

1.  Brainstorm at least 5 different ways to assess the typical delay characteristics of a group of flights.
    Consider the following scenarios:

    -   A flight is 15 minutes early 50% of the time, and 15 minutes late 50% of the time.
    -   A flight is always 10 minutes late.
    -   A flight is 30 minutes early 50% of the time, and 30 minutes late 50% of the time.
    -   99% of the time a flight is on time. 1% of the time it's 2 hours late.

    Which do you think is more important: arrival delay or departure delay?

2.  Which destinations show the greatest variation in air speed?

3.  Create a plot to further explore the adventures of EGE.
    Can you find any evidence that the airport moved locations?